In [1]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
import numpy as np

In [2]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('president_polls.csv')

# Display the first few rows of the DataFrame to verify the import
df.head()

/tmp/ipykernel_76441/615323408.py:2: DtypeWarning: Columns (3,16,17,26,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('president_polls.csv')


,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,hypothetical,party,answer,candidate_id,candidate_name,pct
0,88970,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,general,False,False,NaN,False,DEM,Harris,16661,Kamala Harris,47.0
1,88970,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,general,False,False,NaN,False,REP,Trump,16651,Donald Trump,48.0
2,88976,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,general,False,False,NaN,False,DEM,Harris,16661,Kamala Harris,47.0
3,88976,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,general,False,False,NaN,False,REP,Trump,16651,Donald Trump,48.0
4,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,general,False,False,NaN,False,DEM,Harris,16661,Kamala Harris,48.0


In [3]:
df_state_na = df[df['state'].isna()]

df_state_not_na = df[df['state'].notna()]

print(df_state_na.head())
print(df_state_not_na.columns)
print(df_state_not_na.head())

    poll_id  pollster_id pollster sponsor_ids sponsors   display_name  \
4     88979          770     TIPP         NaN      NaN  TIPP Insights   
5     88979          770     TIPP         NaN      NaN  TIPP Insights   
6     88962          770     TIPP         NaN      NaN  TIPP Insights   
7     88962          770     TIPP         NaN      NaN  TIPP Insights   
49    88958          770     TIPP         NaN      NaN  TIPP Insights   

    pollster_rating_id pollster_rating_name  numeric_grade  pollscore  ...  \
4                  144        TIPP Insights            1.8       -0.4  ...   
5                  144        TIPP Insights            1.8       -0.4  ...   
6                  144        TIPP Insights            1.8       -0.4  ...   
7                  144        TIPP Insights            1.8       -0.4  ...   
49                 144        TIPP Insights            1.8       -0.4  ...   

      stage  nationwide_batch ranked_choice_reallocated ranked_choice_round  \
4   general  

Let's filter out non scored national pollsters 

In [4]:
df_state_na_clean = df_state_na[df_state_na['numeric_grade'].notna()]
print(df_state_na_clean['numeric_grade'])
print(df_state_na_clean['numeric_grade'].max())

4        1.8
5        1.8
6        1.8
7        1.8
49       1.8
        ... 
16466    2.8
16467    2.8
16468    2.8
16469    2.8
16470    2.8
Name: numeric_grade, Length: 7202, dtype: float64
3.0


Let's create a weight for numerically graded pollsters

In [5]:
# Make a copy to avoid the warning
df_state_na_clean = df_state_na_clean.copy()

# Now safely create the 'weight_score' column
df_state_na_clean.loc[:, 'weight_score'] = df_state_na_clean['numeric_grade'] / 3.0


Let's check weight_grade column

In [6]:
df_state_na_clean.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,hypothetical,party,answer,candidate_id,candidate_name,pct,weight_score
4,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,False,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6
5,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,False,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.6
6,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,False,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6
7,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,False,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.6
49,88958,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,False,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6


Let's go ahead and examine unique methodologies

In [7]:
print(df_state_na_clean['methodology'].unique())

['Online Panel' 'IVR/Online Panel/Text-to-Web' 'Live Phone'
 'Probability Panel' 'Live Phone/Text-to-Web' 'Online Ad'
 'Online Panel/Text-to-Web' 'Text-to-Web/Online Ad' 'IVR/Text' nan
 'Live Phone/Online Panel/Text-to-Web' 'Live Phone/Online Panel/App Panel'
 'Live Phone/Online Panel/Text' 'Live Phone/Probability Panel'
 'Online Panel/Probability Panel' 'IVR' 'Live Phone/Online Panel'
 'Live Phone/Text-to-Web/Online Ad' 'IVR/Text-to-Web'
 'Live Phone/Text/Online Panel' 'IVR/Online Panel' 'Text'
 'Online Panel/Online Ad' 'IVR/Online Panel/Email'
 'IVR/Live Phone/Text/Online Panel/Email' 'Live Phone/Text/Online Ad'
 'Online Panel/Text-to-Web/Text' 'Live Phone/Text-to-Web/App Panel'
 'App Panel' 'IVR/Online Panel/Text-to-Web/Email']


mapping different methodologies to weight_mode

In [8]:
# Make a copy to avoid the warning
df_state_na_clean = df_state_na_clean.copy()
# Mapping the weights to modes based on the table above
mode_weights = {
    'Live Phone': 1.00,
    'Live Phone/Probability Panel': 0.95,
    'Live Phone/Online Panel/Text-to-Web': 0.90,
    'Live Phone/Online Panel/Text': 0.90,
    'Live Phone/Text-to-Web/App Panel': 0.82,
    'Live Phone/Text-to-Web/Online Ad': 0.85,
    'Live Phone/Text-to-Web': 0.85,
    'Live Phone/Text/Online Panel': 0.90,
    'Live Phone/Online Panel': 0.85,
    'Live Phone/Online Panel/App Panel': 0.85,
    'IVR/Live Phone/Text/Online Panel/Email': 0.80,
    'Live Phone/Text/Online Ad': 0.80,
    'IVR/Online Panel/Email': 0.77,
    'IVR/Online Panel/Text-to-Web/Email': 0.75,
    'IVR/Online Panel/Text-to-Web': 0.75,
    'IVR/Online Panel': 0.70,
    'IVR': 0.70,
    'Online Panel/Probability Panel': 0.65,
    'Probability Panel': 0.65,
    'Online Panel/Text-to-Web': 0.60,
    'Online Panel/Online Ad': 0.55,
    'Online Panel': 0.50,
    'Online Ad': 0.50,
    'App Panel': 0.50,
    'Online Panel/Text-to-Web/Text': 0.50,
    'IVR/Text-to-Web': 0.50,
    'Text-to-Web/Online Ad': 0.45,
    'Text': 0.40,
    'IVR/Text': 0.40,
    'nan' : 0.50,
     np.nan: 0.50  # Handling missing or unknown values
}

# Apply the mapping to create a new column 'weight_mode'
df_state_na_clean.loc[:,'weight_mode'] = df_state_na_clean['methodology'].map(mode_weights)

Let's check out the 'weight_mode' column

In [9]:
print(df_state_na_clean.head())

    poll_id  pollster_id pollster sponsor_ids sponsors   display_name  \
4     88979          770     TIPP         NaN      NaN  TIPP Insights   
5     88979          770     TIPP         NaN      NaN  TIPP Insights   
6     88962          770     TIPP         NaN      NaN  TIPP Insights   
7     88962          770     TIPP         NaN      NaN  TIPP Insights   
49    88958          770     TIPP         NaN      NaN  TIPP Insights   

    pollster_rating_id pollster_rating_name  numeric_grade  pollscore  ...  \
4                  144        TIPP Insights            1.8       -0.4  ...   
5                  144        TIPP Insights            1.8       -0.4  ...   
6                  144        TIPP Insights            1.8       -0.4  ...   
7                  144        TIPP Insights            1.8       -0.4  ...   
49                 144        TIPP Insights            1.8       -0.4  ...   

   ranked_choice_reallocated  ranked_choice_round hypothetical party  answer  \
4           

Let's create a weight for sample size, but first let's look for NaN in sample_size column

In [10]:
# Count the number of NaN values in the 'sample_size' column
nan_count = df['sample_size'].isna().sum()

print(f"Number of NaN values in 'sample_size': {nan_count}")

# Calculate the mean of the available (non-NaN) sample sizes
mean_sample_size = df['sample_size'].mean()

print(f"Mean of available sample sizes: {mean_sample_size}")


Number of NaN values in 'sample_size': 143
Mean of available sample sizes: 1595.367728107777


In [11]:
import numpy as np

In [12]:
# Step 2: Create the 'weight_sample' column
df_state_na_clean['weight_sample'] = df_state_na_clean['sample_size'].apply(lambda x: np.sqrt(x) if not np.isnan(x) else np.sqrt(mean_sample_size))

# Display the first few rows to verify
df_state_na_clean.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,ranked_choice_round,hypothetical,party,answer,candidate_id,candidate_name,pct,weight_score,weight_mode,weight_sample
4,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,35.888717
5,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.6,0.5,35.888717
6,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,35.944402
7,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.6,0.5,35.944402
49,88958,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,36.510273


Sort end_date values in descending order

In [13]:
# Convert 'end_date' to datetime format with specified format for single/double digits in month/day
df_state_na_clean['end_date'] = pd.to_datetime(df_state_na_clean['end_date'], format='%m/%d/%y', errors='coerce')

# Sort the DataFrame by 'end_date'
df_state_na_clean_sorted = df_state_na_clean.sort_values(by='end_date',ascending=False)

In [14]:
df_state_na_clean_sorted.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,ranked_choice_round,hypothetical,party,answer,candidate_id,candidate_name,pct,weight_score,weight_mode,weight_sample
4,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,35.888717
5,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.6,0.5,35.888717
6,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,35.944402
7,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.6,0.5,35.944402
49,88958,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,NaN,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,36.510273


Let's create a 'days_past_index' that can be used for weight_time_decay value for moving average

In [15]:
# Step 3: Get the first (top) date after sorting
first_date = df_state_na_clean_sorted['end_date'].iloc[0]

# Step 4: Compute the difference in days and create the 'days_past_index' column
df_state_na_clean_sorted['days_past_index'] = (first_date - df_state_na_clean_sorted['end_date']).dt.days

In [16]:
df_state_na_clean_sorted.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,hypothetical,party,answer,candidate_id,candidate_name,pct,weight_score,weight_mode,weight_sample,days_past_index
4,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,35.888717,0
5,88979,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,REP,Trump,16651,Donald Trump,48.0,0.6,0.5,35.888717,0
6,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,35.944402,1
7,88962,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,REP,Trump,16651,Donald Trump,48.0,0.6,0.5,35.944402,1
49,88958,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,False,DEM,Harris,16661,Kamala Harris,48.0,0.6,0.5,36.510273,2


In [17]:
df_state_na_clean_sorted.tail()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,hypothetical,party,answer,candidate_id,candidate_name,pct,weight_score,weight_mode,weight_sample,days_past_index
16466,74812,241,Ipsos,71.0,Reuters,Ipsos,154,Ipsos,2.8,-0.9,...,True,REP,Cruz,16641,Ted Cruz,24.0,0.933333,0.65,33.241540,1290
16467,74812,241,Ipsos,71.0,Reuters,Ipsos,154,Ipsos,2.8,-0.9,...,True,DEM,Biden,19368,Joe Biden,41.0,0.933333,0.65,33.256578,1290
16468,74812,241,Ipsos,71.0,Reuters,Ipsos,154,Ipsos,2.8,-0.9,...,True,REP,DeSantis,16646,Ron DeSantis,25.0,0.933333,0.65,33.256578,1290
16469,74812,241,Ipsos,71.0,Reuters,Ipsos,154,Ipsos,2.8,-0.9,...,True,DEM,Biden,19368,Joe Biden,44.0,0.933333,0.65,33.271610,1290
16470,74812,241,Ipsos,71.0,Reuters,Ipsos,154,Ipsos,2.8,-0.9,...,True,REP,Haley,16640,Nikki Haley,19.0,0.933333,0.65,33.271610,1290


In [18]:
# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_na_clean_sorted_cutoff = df_state_na_clean_sorted[df_state_na_clean_sorted['end_date'] >= cutoff_date].copy()

# Check for NaN values in 'weight_mode', 'weight_sample', and 'weight_score'
nan_check = df_state_na_clean_sorted_cutoff[['weight_mode', 'weight_sample', 'weight_score']].isna().sum()

# Step 2: Check for infinite values in the involved columns
inf_check = df_state_na_clean_sorted_cutoff[['weight_mode', 'weight_sample', 'weight_score']].isin([np.inf, -np.inf]).sum()
print(f"Number of infinite values:\n{inf_check}")


# Print the result to verify if there are any NaN values
print(nan_check)

# Step 1: Filter rows where 'weight_mode' is NaN
nan_weight_mode = df_state_na_clean_sorted_cutoff[df_state_na_clean_sorted_cutoff['weight_mode'].isna()]

# Step 2: Display the 'methodology' or other relevant columns to investigate the methodology used
# For example, we'll check 'pollster', 'sponsors', and 'methodology' (if available) along with 'weight_mode'
nan_weight_mode_info = nan_weight_mode[['pollster', 'sponsors', 'methodology', 'weight_mode']]

# Print the resulting DataFrame for verification
print(nan_weight_mode_info)

Number of infinite values:
weight_mode      0
weight_sample    0
weight_score     0
dtype: int64
weight_mode      0
weight_sample    0
weight_score     0
dtype: int64
Empty DataFrame
Columns: [pollster, sponsors, methodology, weight_mode]
Index: []


Let's compute a weight_i where weight_i is for a given 'end_date' the determined index weight which determines for a given end_date the weight assigned to a row used in computing the point average_i. Note: we will use the sorted dates to filter dates that only on the date or before, then we will compute the 'weight_time' using the formula: exp(-lambdat) where lambda = 1.0 and t = days elapsed since the beginning of the poll end date.
finally we can compute the weight_i = 'weight_mode''weight_sample'*weight_score'*weight_time'

In [19]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = 1.0
avg_window = 9
weighted_averages = []

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_na_clean_sorted_cutoff = df_state_na_clean_sorted[df_state_na_clean_sorted['end_date'] >= cutoff_date].copy()
df_lv = df_state_na_clean_sorted_cutoff[df_state_na_clean_sorted_cutoff['population']=='lv'].copy()

# Iterate through each unique end date
for current_date in df_lv['end_date'].unique():
    
    # Step 2: Filter the data for polls on or before the current end date
    current_data = df_lv[df_lv['end_date'] <= current_date].copy()
    
    # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
    current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
    
    # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)
    current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])

    # Check for NaN entries in the 'weight_time' column
    #nan_weight_time = current_data['weight_time'].isna().sum()
    
    # Print the result
    #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")

    # Step 2: Check for infinite values in the involved columns
    #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
    #print(f"Number of infinite values:\n{inf_check}")
    
    # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
    for candidate in candidate_ids:
        #  We select H2H if avaible and only use non H2H if H2H is not available
        candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
        # Step 1: Group by 'created_at' and 'question_id' and count occurrences
        grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
        
        # Step 2: Initialize an empty DataFrame to store results
        result_df = pd.DataFrame()
    
        # Step 3: Iterate through each 'created_at' date group
        for date, group in grouped.groupby('created_at'):
            # Step 4: Filter for question_ids with count == 2
            question_ids_with_count_2 = group[group['count'] == 2]
            
            if not question_ids_with_count_2.empty:
                # If question_ids with count 2 exist, include only them
                selected_question_ids = question_ids_with_count_2['question_id']
            else:
                # If no question_ids with count 2 exist, include all question_ids for that date
                selected_question_ids = group['question_id']
            
            # Step 5: Filter the original DataFrame to include only selected question_ids for that date
            filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
            
            # Step 6: Append the filtered DataFrame to the result
            result_df = pd.concat([result_df, filtered_df])
        candidate_data = result_df.copy()
        
        # Step 7: Filter data for the specific candidate
        
        c_mean = candidate_data[candidate_data['days_past_index']< 30]['pct'].mean() #gather mean for past 30 days of likely voters
        c_std =  candidate_data[candidate_data['days_past_index']< 30]['pct'].std()  #gather standard deviation for past 30 days of likely voters
        candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
        candidate_data['weight_outlier'] = np.exp(-1.0*candidate_data['zscores'])              
        # Step 8: Compute the total weight 'w_i' for each poll
        #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
        candidate_data.loc[:, 'w_i'] = (
            candidate_data['weight_mode'] * candidate_data['weight_sample'] *
            candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']
        )
        #Start House effect data computation#
         # Create an empty dictionary to store the results
        pollster_means = {}
        
        # Group the data by 'pollster_id'
        grouped = candidate_data.groupby('pollster_id')
        
        # Loop through each group (i.e., each pollster)
        for pollster_id, group in grouped:
            # Find the most recent 'end_date' for this pollster based on elapsed time (min days_past_index)
            last_end_date_index = group['days_past_index'].min()  # Smallest elapsed time is most recent
            
            # Calculate the start date for the filtering window (from last_end_date + avg_window days)
            start_date_index = last_end_date_index + avg_window  # Adjust window back from the most recent
            
            # Filter the group's data to only include rows between start_date and last_end_date
            filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
            
            # Calculate the mean of 'pct' for this filtered group (pollster-specific mean)
            pollster_mean = filtered_group['pct'].mean()
        
            # Calculate the overall mean ('pct') for all data within this window for all pollsters
            c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
            c_mean2 = c_mean2data['pct'].mean()  # The overall average within the same window
        
            # Calculate the house effect (difference between pollster mean and overall mean)
            house_effect = pollster_mean - c_mean2
            
            # Store the result in the dictionary
            pollster_means[pollster_id] = house_effect
        
        # Convert the dictionary to a pandas Series for easier handling
        c_mean_by_pollster = pd.Series(pollster_means)
        
        # Convert to a DataFrame for easier manipulation
        c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
        
        # Rename the columns for clarity
        c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
        
        # Merge the house effect back into the original data
        candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
        
        candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
        #print(candidate_data.head(10))
        # Apply the house effect to adjust the 'pct' values
        candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
        #print(candidate_data['adjusted_pct'].head(10))
        # Step 8: Compute the total weight 'w_i' for each poll
        #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
        candidate_data.loc[:, 'w_i'] = (
            candidate_data['weight_mode'] * candidate_data['weight_sample'] *
            candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']
        )
        # check for NaN entries in the w_i column
        #nan_weight_i = candidate_data['w_i'].isna().sum()
        #print result
        #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
        
        # Step 3: Identify rows with NaN in w_i to examine their individual values
        #nan_rows = candidate_data[candidate_data['w_i'].isna()]
        #print("Rows producing NaN in 'w_i':")
        #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
        
        # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
        weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
        
        # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
        if not candidate_data.empty:
            candidate_name = candidate_data['candidate_name'].iloc[0]
            weighted_averages.append({
                'end_date': current_date,
                'candidate_id': candidate,
                'candidate_name': candidate_name,
                'weighted_average_pct': weighted_average_pct
            })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
df_weighted_averages.head()

,end_date,candidate_id,candidate_name,weighted_average_pct
0,2024-10-27,16661,Kamala Harris,48.473593
1,2024-10-27,16651,Donald Trump,47.517388
2,2024-10-26,16661,Kamala Harris,48.691350
3,2024-10-26,16651,Donald Trump,47.647597
4,2024-10-25,16661,Kamala Harris,48.900022


Let's view the plot with weight average:

In [20]:
# Create the line chart with points
# Create a customized color encoding for Trump and Harris
color_scale = alt.Scale(domain=['Donald Trump', 'Kamala Harris'], range=['red', 'blue'])

# Create the line chart with points, assigning specific colors to each candidate
chart = alt.Chart(df_weighted_averages).mark_line(point=True).encode(
    x='end_date:T',  # Temporal encoding for end_date
    y='weighted_average_pct:Q',  # Quantitative encoding for weighted averages
    color=alt.Color('candidate_name:N', scale=color_scale),  # Custom color scale for candidates
    tooltip=['end_date', 'candidate_name', 'weighted_average_pct']  # Add tooltips to show details
).properties(
    title='Weighted Average Polling Results Over Time',
    width=600,
    height=400
)

chart.show()


alt.Chart(...)

In [21]:
# Pivot the data so that we have both candidates' weighted averages in the same row for each end_date
df_pivot = df_weighted_averages.pivot(index='end_date', columns='candidate_name', values='weighted_average_pct').reset_index()

# Create the line chart with points, showing both candidates' weighted percentages in the tooltip
chart = alt.Chart(df_pivot).transform_fold(
    ['Donald Trump', 'Kamala Harris'],  # Specify the candidates to include
    as_=['candidate', 'weighted_average_pct']  # Assign new names for the folded fields
).mark_line(point=True).encode(
    x='end_date:T',  # Temporal encoding for end_date
    y='weighted_average_pct:Q',  # Quantitative encoding for weighted averages
    color=alt.Color('candidate:N', scale=alt.Scale(domain=['Donald Trump', 'Kamala Harris'], range=['red', 'blue'])),  # Custom colors
    tooltip=['end_date:T', 'Kamala Harris:Q','Donald Trump:Q']  # Tooltip to display both candidates' percentages
).properties(
    title='Weighted Average Polling Results for Trump and Harris Over Time',
    width=600,
    height=400
)

chart.show()

alt.Chart(...)

Let's look at the loess smoothed curve of this

In [22]:



# Define the color scale for the two candidates
color_scale = alt.Scale(domain=['Donald Trump', 'Kamala Harris'], range=['red', 'blue'])

# Step 1: Create a selection brush for the horizontal axis (end_date)
brush = alt.selection_interval(encodings=['x'])  # Brush selection on the x-axis

# Step 2: Create the base chart with points and LOESS smoothing (Chart 1)
base = alt.Chart(df_weighted_averages).mark_point().encode(
    x='end_date:T',
    y='weighted_average_pct:Q',
    color=alt.Color('candidate_name:N', scale=color_scale)
)

loess = base.transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name'], bandwidth=0.3
).mark_line()

chart1 = (base + loess).add_params(
    brush  # Add the brush selection to Chart 1
).properties(
    title='Weighted Average Polling Results with LOESS Smoothing (Selectable)',
    width=600,
    height=400
)

# Step 3: Create the side-by-side chart that will show the selected region with tooltip

# Apply the same selection filter to both points and LOESS in chart2
selected_points = base.encode(
    opacity=alt.condition(brush, alt.value(1), alt.value(0)),  # Highlight only selected points
    tooltip=['end_date:T', 'candidate_name:N', 'weighted_average_pct:Q']  # Add tooltip for points
).transform_filter(
    brush  # Filter by brush selection
)

# Apply LOESS smoothing for the selected range
selected_loess = alt.Chart(df_weighted_averages).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name'], bandwidth=0.3
).mark_line().encode(
    x='end_date:T',
    y='weighted_average_pct:Q',
    color=alt.Color('candidate_name:N', scale=color_scale),
    tooltip=['end_date:T', 'candidate_name:N', 'weighted_average_pct:Q']
).transform_filter(
    brush  # Filter the LOESS line by the brush selection
)

# Combine selected points and LOESS for the second chart
chart2 = (selected_points + selected_loess).properties(
    title='Selected Region with LOESS Smoothing',
    width=600,
    height=400
)

# Combine the two charts side by side
final_chart = alt.hconcat(chart1, chart2)

# Display the final chart
final_chart.show()


alt.HConcatChart(...)

Let's look at A/B graded pollsters

In [23]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = 0.23
weighted_averages = []

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_na_clean_sorted_cutoff = df_state_na_clean_sorted[df_state_na_clean_sorted['end_date'] >= cutoff_date].copy()
df_ab_pollsters = df_state_na_clean_sorted_cutoff[df_state_na_clean_sorted_cutoff['numeric_grade'] >= 2.4].copy()
df_lv = df_ab_pollsters[df_ab_pollsters['population'] == 'lv'].copy()
# Iterate through each unique end date
for current_date in df_lv['end_date'].unique():
    
    # Step 2: Filter the data for polls on or before the current end date
    current_data = df_lv[df_lv['end_date'] <= current_date].copy()
    
    # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
    current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
    
    # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)
    current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])

    # Check for NaN entries in the 'weight_time' column
    #nan_weight_time = current_data['weight_time'].isna().sum()
    
    # Print the result
    #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")

    # Step 2: Check for infinite values in the involved columns
    #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
    #print(f"Number of infinite values:\n{inf_check}")
    
    # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
    for candidate in candidate_ids:
        #  We select H2H if avaible and only use non H2H if H2H is not available
        candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
        # Step 1: Group by 'created_at' and 'question_id' and count occurrences
        grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
        
        # Step 2: Initialize an empty DataFrame to store results
        result_df = pd.DataFrame()
    
        # Step 3: Iterate through each 'created_at' date group
        for date, group in grouped.groupby('created_at'):
            # Step 4: Filter for question_ids with count == 2
            question_ids_with_count_2 = group[group['count'] == 2]
            
            if not question_ids_with_count_2.empty:
                # If question_ids with count 2 exist, include only them
                selected_question_ids = question_ids_with_count_2['question_id']
            else:
                # If no question_ids with count 2 exist, include all question_ids for that date
                selected_question_ids = group['question_id']
            
            # Staep 5: Filter the original DataFrame to include only selected question_ids for that date
            filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
            
            # Step 6: Append the filtered DataFrame to the result
            result_df = pd.concat([result_df, filtered_df])
        candidate_data = result_df.copy() 
        
        # Step 7: Filter data for the specific candidate
        
        c_mean = candidate_data[candidate_data['days_past_index']< 40]['pct'].mean() #gather mean for past 30 days of likely voters
        c_std =  candidate_data[candidate_data['days_past_index']< 40]['pct'].std()  #gather standard deviation for past 30 days of likely voters
        candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
        candidate_data['weight_outlier'] = np.exp(-1.0*candidate_data['zscores'])              
        # Step 8: Compute the total weight 'w_i' for each poll
        #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
        candidate_data.loc[:, 'w_i'] = (
            candidate_data['weight_mode'] * candidate_data['weight_sample'] *
            candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']
        )

        #Start House effect data computation#
         # Create an empty dictionary to store the results
        pollster_means = {}
        
        # Group the data by 'pollster_id'
        grouped = candidate_data.groupby('pollster_id')
        
        # Loop through each group (i.e., each pollster)
        for pollster_id, group in grouped:
            # Find the most recent 'end_date' for this pollster based on elapsed time (min days_past_index)
            last_end_date_index = group['days_past_index'].min()  # Smallest elapsed time is most recent
            
            # Calculate the start date for the filtering window (from last_end_date + avg_window days)
            start_date_index = last_end_date_index + avg_window  # Adjust window back from the most recent
            
            # Filter the group's data to only include rows between start_date and last_end_date
            filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
            
            # Calculate the mean of 'pct' for this filtered group (pollster-specific mean)
            pollster_mean = filtered_group['pct'].mean()
        
            # Calculate the overall mean ('pct') for all data within this window for all pollsters
            c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
            c_mean2 = c_mean2data['pct'].mean()  # The overall average within the same window
        
            # Calculate the house effect (difference between pollster mean and overall mean)
            house_effect = pollster_mean - c_mean2
            
            # Store the result in the dictionary
            pollster_means[pollster_id] = house_effect
        
        # Convert the dictionary to a pandas Series for easier handling
        c_mean_by_pollster = pd.Series(pollster_means)
        
        # Convert to a DataFrame for easier manipulation
        c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
        
        # Rename the columns for clarity
        c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
        
        # Merge the house effect back into the original data
        candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
        
        candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
        #print(candidate_data.head(10))
        # Apply the house effect to adjust the 'pct' values
        candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
        ## finish house effect weighting  ##   
        
        # Step 8: Compute the total weight 'w_i' for each poll
        #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
        candidate_data.loc[:, 'w_i'] = (
            candidate_data['weight_mode'] * candidate_data['weight_sample'] *
            candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']
        )
        # check for NaN entries in the w_i column
        #nan_weight_i = candidate_data['w_i'].isna().sum()
        #print result
        #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
        
        # Step 3: Identify rows with NaN in w_i to examine their individual values
        #nan_rows = candidate_data[candidate_data['w_i'].isna()]
        #print("Rows producing NaN in 'w_i':")
        #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
        
        # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
        weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
        
        # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
        if not candidate_data.empty:
            candidate_name = candidate_data['candidate_name'].iloc[0]
            weighted_averages.append({
                'end_date': current_date,
                'candidate_id': candidate,
                'candidate_name': candidate_name,
                'weighted_average_pct': weighted_average_pct
            })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
df_weighted_averages.head()



,end_date,candidate_id,candidate_name,weighted_average_pct
0,2024-10-25,16661,Kamala Harris,48.517298
1,2024-10-25,16651,Donald Trump,47.410679
2,2024-10-24,16661,Kamala Harris,48.481471
3,2024-10-24,16651,Donald Trump,47.322653
4,2024-10-23,16661,Kamala Harris,48.560670


In [24]:
# Pivot the data so that we have both candidates' weighted averages in the same row for each end_date
df_pivot = df_weighted_averages.pivot(index='end_date', columns='candidate_name', values='weighted_average_pct').reset_index()

# Create the line chart with points, showing both candidates' weighted percentages in the tooltip
chart = alt.Chart(df_pivot).transform_fold(
    ['Donald Trump', 'Kamala Harris'],  # Specify the candidates to include
    as_=['candidate', 'weighted_average_pct']  # Assign new names for the folded fields
).mark_line(point=True).encode(
    x='end_date:T',  # Temporal encoding for end_date
    y='weighted_average_pct:Q',  # Quantitative encoding for weighted averages
    color=alt.Color('candidate:N', scale=alt.Scale(domain=['Donald Trump', 'Kamala Harris'], range=['red', 'blue'])),  # Custom colors
    

tooltip=['end_date:T', 'Kamala Harris:Q','Donald Trump:Q'] 

 # Tooltip to display both candidates' percentages
).properties(
    title='Weighted Average Polling Results for Trump and Harris Over Time',
    width=600,
    height=400
)

chart.show()

alt.Chart(...)

In [25]:



# Define the color scale for the two candidates
color_scale = alt.Scale(domain=['Donald Trump', 'Kamala Harris'], range=['red', 'blue'])

# Step 1: Create a selection brush for the horizontal axis (end_date)
brush = alt.selection_interval(encodings=['x'])  # Brush selection on the x-axis

# Step 2: Create the base chart with points and LOESS smoothing (Chart 1)
base = alt.Chart(df_weighted_averages).mark_point().encode(
    x='end_date:T',
    y='weighted_average_pct:Q',
    color=alt.Color('candidate_name:N', scale=color_scale)
)

loess = base.transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name'], bandwidth=0.3
).mark_line()

chart1 = (base + loess).add_params(
    brush  # Add the brush selection to Chart 1
).properties(
    title='Weighted Average Polling Results with LOESS Smoothing (Selectable)',
    width=600,
    height=400
)

# Step 3: Create the side-by-side chart that will show the selected region with tooltip

# Apply the same selection filter to both points and LOESS in chart2
selected_points = base.encode(
    opacity=alt.condition(brush, alt.value(1), alt.value(0)),  # Highlight only selected points
    tooltip=['end_date:T', 'candidate_name:N', 'weighted_average_pct:Q']  # Add tooltip for points
).transform_filter(
    brush  # Filter by brush selection
)

# Apply LOESS smoothing for the selected range
selected_loess = alt.Chart(df_weighted_averages).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name'], bandwidth=0.3
).mark_line().encode(
    x='end_date:T',
    y='weighted_average_pct:Q',
    color=alt.Color('candidate_name:N', scale=color_scale),
    tooltip=['end_date:T', 'candidate_name:N', 'weighted_average_pct:Q']
).transform_filter(
    brush  # Filter the LOESS line by the brush selection
)

# Combine selected points and LOESS for the second chart
chart2 = (selected_points + selected_loess).properties(
    title='Selected Region with LOESS Smoothing',
    width=600,
    height=400
)

# Combine the two charts side by side
final_chart = alt.hconcat(chart1, chart2)

# Display the final chart
final_chart.show()


alt.HConcatChart(...)

### Moving on to Battleground state data

Let's clean 'numeric_grade' rows for df_state_not_na so that there aren't ungraded pollsters in our list

In [26]:
#df_state_not_na_clean = df_state_not_na[df_state_not_na['numeric_grade'].notna()]

In [27]:
#na_num = df_state_not_na_clean['numeric_grade'].isna().sum()
#print(na_num)

Let's normalize 'numeric_grade' to create a 'weight_score'

In [28]:
# Make a copy to avoid the warning
df_state_not_na = df_state_not_na.copy()

# Step 1: Replace NaN values in 'numeric_grade' with a low grade (e.g., 1.0)
df_state_not_na['numeric_grade'].fillna(0.5, inplace=True)

df_state_not_na_clean = df_state_not_na.copy()
# Step 2: Safely create the 'weight_score' column by dividing 'numeric_grade' by 3.0
df_state_not_na_clean.loc[:, 'weight_score'] = df_state_not_na_clean['numeric_grade'] / 3.0

# Verify the updated DataFrame
print(df_state_not_na_clean[['numeric_grade', 'weight_score']].head())

   numeric_grade  weight_score
0            2.0      0.666667
1            2.0      0.666667
2            2.0      0.666667
3            2.0      0.666667
8            0.7      0.233333


/tmp/ipykernel_76441/731753424.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_state_not_na['numeric_grade'].fillna(0.5, inplace=True)


In [29]:
na_num = df_state_not_na_clean['weight_score'].isna().sum()
print(na_num)

0


#### Creating 'weight_mode' weight for methodology

In [30]:
import numpy as np
# Make a copy to avoid the warning
df_state_not_na_clean = df_state_not_na_clean.copy()
# Mapping the weights to modes based on the table above
mode_weights = {
    "Text-to-Web/Email": 0.75,
    "IVR/Live Phone/Text-to-Web": 0.76,
    "IVR/Live Phone/Online Panel": 0.78,
    "IVR/Live Phone/Online Panel/Text-to-Web": 0.77,
    "Live Phone/Text-to-Web/Email/Mail-to-Web": 0.76,
    "Live Phone/Text-to-Web/Email": 0.78,
    "Email/Online Ad": 0.73,
    "Online Panel/Email": 0.78,
    "Live Phone/Online Panel/Mail-to-Web": 0.78,
    "IVR/Text-to-Web/Email": 0.72,
    'Email':0.8,
    'Live Phone': 1.00,
    'Live Phone/Probability Panel': 0.95,
    'Live Phone/Online Panel/Text-to-Web': 0.90,
    'Live Phone/Online Panel/Text': 0.90,
    'Live Phone/Text-to-Web/App Panel': 0.85,
    'Live Phone/Text-to-Web/Online Ad': 0.85,
    'Live Phone/Text-to-Web': 0.85,
    'Live Phone/Text/Online Panel': 0.90,
    'Live Phone/Online Panel': 0.85,
    'Live Phone/Online Panel/App Panel': 0.85,
    'Live Phone/Text-to-Web/Email/Mail-to-Web/Mail-to-Phone':0.76,
    'Live Phone/Email':0.82,
    'Live Phone/Online Panel/Text-to-Web/Text':0.8,
    'Live Phone/Text':0.83,
    'IVR/Live Phone/Text/Online Panel/Email': 0.80,
    'Live Phone/Text/Online Ad': 0.80,
    'IVR/Live Phone/Text':0.78,
    'IVR/Online Panel/Email': 0.77,
    'IVR/Online Panel/Text-to-Web/Email': 0.75,
    'IVR/Online Panel/Text-to-Web': 0.75,
    'IVR/Online Panel': 0.70,
    'IVR': 0.70,
    'Mail-to-Web/Mail-to-Phone': 0.7,
    'Online Panel/Probability Panel': 0.65,
    'Probability Panel': 0.65,
    'Online Panel/Email/Text-to-Web':0.77,
    'Online Panel/Text-to-Web': 0.60,
    'Online Panel/Text':0.78,
    'Online Panel/Online Ad': 0.55,
    'Online Panel': 0.50,
    'Online Ad': 0.50,
    'App Panel': 0.50,
    'Online Panel/Text-to-Web/Text': 0.50,
    'IVR/Text-to-Web': 0.50,
    'Text-to-Web/Online Ad': 0.45,
    'Text-to-Web':0.45,
    'Text': 0.40,
    'IVR/Text': 0.40,
    'nan' : 0.50,
     np.nan: 0.50  # Handling missing or unknown values
}

# Apply the mapping to create a new column 'weight_mode'
df_state_not_na_clean.loc[:,'weight_mode'] = df_state_not_na_clean['methodology'].map(mode_weights)

In [31]:
num_na = df_state_not_na_clean['weight_mode'].isna().sum()
print(num_na)
df_ret = df_state_not_na_clean[df_state_not_na_clean['weight_mode'].isna()][['methodology']]
print(df_ret['methodology'].unique())

2
['Mail-to-Web']


#### Create a Weight_Sample weight for sample size

In [32]:
# Step 2: Create the 'weight_sample' column
df_state_not_na_clean['weight_sample'] = df_state_not_na_clean['sample_size'].apply(lambda x: np.sqrt(x/600) if not np.isnan(x) else np.sqrt(mean_sample_size/600))

# Display the first few rows to verify
df_state_not_na_clean.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,ranked_choice_round,hypothetical,party,answer,candidate_id,candidate_name,pct,weight_score,weight_mode,weight_sample
0,88970,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,DEM,Harris,16661,Kamala Harris,47.0,0.666667,0.5,1.154701
1,88970,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.666667,0.5,1.154701
2,88976,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,DEM,Harris,16661,Kamala Harris,47.0,0.666667,0.5,1.154701
3,88976,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.666667,0.5,1.154701
8,88967,1250,Trafalgar Group,NaN,NaN,Trafalgar Group,338,Trafalgar Group,0.7,0.6,...,NaN,False,DEM,Harris,16661,Kamala Harris,45.9,0.233333,0.8,1.345982


In [33]:
num_na = df_state_not_na_clean['weight_sample'].isna().sum()
print(num_na)

0


####  Converting 'end_date' to datetime format and sorting dataframe by end_date

In [34]:
# Convert 'end_date' to datetime format with specified format for single/double digits in month/day
df_state_not_na_clean['end_date'] = pd.to_datetime(df_state_not_na_clean['end_date'], format='%m/%d/%y', errors='coerce')

# Sort the DataFrame by 'end_date'
df_state_not_na_clean_sorted = df_state_not_na_clean.sort_values(by='end_date',ascending=False)

In [35]:
df_state_not_na_clean_sorted.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,ranked_choice_round,hypothetical,party,answer,candidate_id,candidate_name,pct,weight_score,weight_mode,weight_sample
0,88970,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,DEM,Harris,16661,Kamala Harris,47.0,0.666667,0.5,1.154701
3,88976,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.666667,0.5,1.154701
1,88970,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,REP,Trump,16651,Donald Trump,48.0,0.666667,0.5,1.154701
2,88976,235,InsiderAdvantage,NaN,NaN,InsiderAdvantage,243,InsiderAdvantage,2.0,-0.3,...,NaN,False,DEM,Harris,16661,Kamala Harris,47.0,0.666667,0.5,1.154701
39,88973,1424,Siena/NYT,NaN,NaN,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,NaN,False,GRE,Stein,31116,Jill Stein,0.0,1.000000,1.0,1.410674


#### Let's compute battleground state PA

In [80]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Pennsylvania"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90),(91,120)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Pennsylvania")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/3641546870.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Pennsylvania


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-27,16661,Kamala Harris,48.579230,0.05,19
1,2024-10-27,16651,Donald Trump,47.602343,0.05,19
2,2024-10-22,16661,Kamala Harris,48.595568,0.05,19
3,2024-10-22,16651,Donald Trump,47.553198,0.05,19
4,2024-10-20,16661,Kamala Harris,48.586075,0.05,19
5,2024-10-20,16651,Donald Trump,47.565825,0.05,19
6,2024-10-18,16661,Kamala Harris,48.590597,0.05,19
7,2024-10-18,16651,Donald Trump,47.506105,0.05,19
8,2024-10-17,16661,Kamala Harris,48.624641,0.05,19
9,2024-10-17,16651,Donald Trump,47.502687,0.05,19


In [81]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Pennsylvania')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Pennsylvania)"
)

# Display the final chart
final_chart.display()


Pennsylvania
     end_date candidate_name  weighted_average_pct
85 2024-10-27  Kamala Harris             48.454490
84 2024-10-27   Donald Trump             47.991413
83 2024-10-22  Kamala Harris             48.498960
82 2024-10-22   Donald Trump             47.992385
81 2024-10-20  Kamala Harris             48.450353
80 2024-10-20   Donald Trump             47.965543
79 2024-10-18  Kamala Harris             48.400486
78 2024-10-18   Donald Trump             47.709527
77 2024-10-17  Kamala Harris             48.374188
76 2024-10-17   Donald Trump             47.658411


alt.LayerChart(...)

#### Kamala Harris and Trump Forecast in PA using Lowess curve fit data

In [82]:
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np
import datetime

In [83]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive

alt.LayerChart(...)

/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RMSE for Kamala Harris: 0.09006827217414816
Final forecast for Kamala Harris on Nov 5, 2024: 48.29%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


alt.LayerChart(...)

RMSE for Donald Trump: 0.11236673160018205
Final forecast for Donald Trump on Nov 5, 2024: 48.53%

Final Predictions for November 5, 2024:
Kamala Harris: 48.29%
Donald Trump: 48.53%


#### Summary Stats of Pollster House Effects in State

In [40]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
house_effect_summary = he_agg.groupby(['pollster','candidate_name'])['house_effect'].describe()

# Display the summary statistics
print(house_effect_summary)

                                                        count      mean  \
pollster                                candidate_name                    
AtlasIntel                              Donald Trump     14.0  2.941970   
                                        Kamala Harris    14.0 -0.932159   
Beacon/Shaw                             Donald Trump     15.0  1.277028   
                                        Kamala Harris    15.0 -0.804371   
CNN/SSRS                                Donald Trump     27.0  0.304579   
...                                                       ...       ...   
Washington Post/George Mason University Kamala Harris     6.0  0.723333   
Wick                                    Donald Trump     27.0  1.804579   
                                        Kamala Harris    27.0  0.477106   
YouGov                                  Donald Trump     30.0  1.185889   
                                        Kamala Harris    30.0  1.246222   

                        

#### Time Series of Pollster House Effects in State

In [41]:
import altair as alt

# Define selection for highlighting specific pollster
highlight = alt.selection_multi(fields=['pollster'], bind='legend')

# Manually map shapes for the candidates ('circle' for Kamala Harris and 'square' for Donald Trump)
shape_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['circle', 'square'])

# Scatter plot for house effect data with shape highlight based on candidate_name (restricted to circle and square)
scatter = alt.Chart(he_agg).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('house_effect:Q', title='House Effect'),
    color=alt.condition(highlight, 'pollster:N', alt.value('lightgray'), legend=alt.Legend(title="Pollster")),  # Color for pollster with legend
    shape=alt.condition(
        highlight, 
        alt.Shape('candidate_name:N', scale=shape_scale),  # Shape encoding based on candidate
        alt.value('circle')  # Default shape for non-selected points
    ),
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.3)),  # Reduce opacity for non-selected points
    tooltip=['pollster', 'candidate_name', 'end_date', 'house_effect']
).properties(
    width=800,
    height=600
).add_selection(
    highlight
)

# LOESS smoothing for each pollster (groupby pollster)
loess = alt.Chart(he_agg).transform_loess(
    'end_date', 'house_effect', groupby=['pollster']
).mark_line(size=3).encode(
    x='end_date:T',
    y='house_effect:Q',
    color=alt.condition(highlight, 'pollster:N', alt.value('lightgray')),  # Color highlighting for LOESS curve
    size=alt.condition(highlight, alt.value(3), alt.value(1)),  # Thicker line for selected pollsters
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.3))  # Reduce opacity for non-selected lines
)

# Layering the scatter plot and LOESS smoothing curve
final_chart = alt.layer(scatter, loess).properties(
    title="House Effect Data with LOESS Smoothing (Highlight Pollster)"
)

# Display the final chart
final_chart.display()


/tmp/ipykernel_76441/1412662728.py:4: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  highlight = alt.selection_multi(fields=['pollster'], bind='legend')
/tmp/ipykernel_76441/1412662728.py:24: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

In [42]:
import altair as alt

# Group by 'end_date' and 'pollster' and calculate the mean of 'house_effect'
grouped_data = he_agg.groupby(['end_date', 'pollster']).agg(
    avg_house_effect=('house_effect', 'mean')
).reset_index()

# Define selection for pollster highlighting
highlight = alt.selection_multi(fields=['pollster'], bind='legend')

# Create a scatter plot for the average house effect, color-coded by 'pollster'
scatter = alt.Chart(grouped_data).mark_circle(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('avg_house_effect:Q', title='Average House Effect'),
    color=alt.condition(
        highlight,  # Highlight selected pollsters
        'pollster:N',  # Color based on pollster when selected
        alt.value('lightgray')  # Gray out when not selected
    ),
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.3)),  # Opacity for non-selected points
    tooltip=['end_date:T', 'avg_house_effect:Q', 'pollster:N']
).properties(
    width=600,
    height=400,
    title="Average House Effect by End Date, Color-coded by Pollster"
).add_selection(
    highlight  # Add the selection to the chart
)

# LOESS smoothing for each pollster (groupby pollster)
loess = alt.Chart(grouped_data).transform_loess(
    'end_date', 'avg_house_effect', groupby=['pollster']
).mark_line().encode(
    x='end_date:T',
    y='avg_house_effect:Q',
    color=alt.condition(
        highlight,  # Highlight LOESS lines for selected pollsters
        'pollster:N',  # Color based on pollster when selected
        alt.value('lightgray')  # Gray out when not selected
    ),
    size=alt.condition(highlight, alt.value(2), alt.value(1)),  # Thicker line for selected pollsters
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.3))  # Opacity for non-selected lines
)

# Layering the scatter plot and LOESS smoothing lines
final_chart = alt.layer(scatter, loess)

# Display the final chart
final_chart.display()


/tmp/ipykernel_76441/2796102410.py:9: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  highlight = alt.selection_multi(fields=['pollster'], bind='legend')
/tmp/ipykernel_76441/2796102410.py:26: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

#### Looking at the Distributions of Pollster House Effects in State

In [43]:
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### Let's compute battleground state Michigan

In [44]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Michigan"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Michigan")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/2407739034.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Michigan


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-27,16661,Kamala Harris,48.105392,0.05,19
1,2024-10-27,16651,Donald Trump,46.976475,0.05,19
2,2024-10-22,16661,Kamala Harris,48.174117,0.05,19
3,2024-10-22,16651,Donald Trump,47.029332,0.05,19
4,2024-10-21,16661,Kamala Harris,48.100212,0.05,19
5,2024-10-21,16651,Donald Trump,47.102622,0.05,19
6,2024-10-20,16661,Kamala Harris,48.145429,0.05,19
7,2024-10-20,16651,Donald Trump,47.173632,0.05,19
8,2024-10-18,16661,Kamala Harris,48.178359,0.05,19
9,2024-10-18,16651,Donald Trump,47.121521,0.05,19


In [45]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Michigan')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Michigan)"
)

# Display the final chart
final_chart.display()


Michigan
     end_date candidate_name  weighted_average_pct
81 2024-10-27  Kamala Harris             48.152954
80 2024-10-27   Donald Trump             47.160518
79 2024-10-22  Kamala Harris             48.138621
78 2024-10-22   Donald Trump             47.032572
77 2024-10-21  Kamala Harris             48.114458
76 2024-10-21   Donald Trump             46.981937
75 2024-10-20  Kamala Harris             47.716903
74 2024-10-20   Donald Trump             47.605791
72 2024-10-18   Donald Trump             47.672589
73 2024-10-18  Kamala Harris             47.682967


alt.LayerChart(...)

#### Kamala Harris and Trump Forecast in MI using Lowess curve fit data

In [46]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RMSE for Kamala Harris: 0.11565075404657274
Final forecast for Kamala Harris on Nov 5, 2024: 48.89%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


alt.LayerChart(...)

RMSE for Donald Trump: 0.09511448505445315
Final forecast for Donald Trump on Nov 5, 2024: 46.33%

Final Predictions for November 5, 2024:
Kamala Harris: 48.89%
Donald Trump: 46.33%


#### Boxplots of House Effects distributions by Pollster and Candidate

In [47]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### Battleground Wisconsin

In [48]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Wisconsin"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Wisconsin")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/3324404400.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Wisconsin


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-23,16661,Kamala Harris,48.570593,0.05,19
1,2024-10-23,16651,Donald Trump,47.808293,0.05,19
2,2024-10-22,16661,Kamala Harris,48.604044,0.05,19
3,2024-10-22,16651,Donald Trump,47.721602,0.05,19
4,2024-10-21,16661,Kamala Harris,48.676195,0.05,19
5,2024-10-21,16651,Donald Trump,47.699175,0.05,19
6,2024-10-20,16661,Kamala Harris,48.872279,0.05,19
7,2024-10-20,16651,Donald Trump,47.510855,0.05,19
8,2024-10-18,16661,Kamala Harris,48.922333,0.05,19
9,2024-10-18,16651,Donald Trump,47.481736,0.05,19


In [49]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Wisconsin')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Wisconsin)"
)

# Display the final chart
final_chart.display()


Wisconsin
     end_date candidate_name  weighted_average_pct
79 2024-10-23  Kamala Harris             48.389705
78 2024-10-23   Donald Trump             47.913970
77 2024-10-22  Kamala Harris             48.465656
76 2024-10-22   Donald Trump             47.817202
75 2024-10-21  Kamala Harris             48.471630
74 2024-10-21   Donald Trump             47.776013
73 2024-10-20  Kamala Harris             48.452996
72 2024-10-20   Donald Trump             47.831812
70 2024-10-18   Donald Trump             47.910419
71 2024-10-18  Kamala Harris             48.433160


alt.LayerChart(...)

#### Kamala Harris and Trump Forecast in WI using Lowess curve fit data

In [50]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive

alt.LayerChart(...)

RMSE for Kamala Harris: 0.14178571372634996
Final forecast for Kamala Harris on Nov 5, 2024: 48.38%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

RMSE for Donald Trump: 0.1307334730438486
Final forecast for Donald Trump on Nov 5, 2024: 47.88%

Final Predictions for November 5, 2024:
Kamala Harris: 48.38%
Donald Trump: 47.88%


#### Boxplots of House Effects distributions by Pollster and Candidate

In [51]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### Battleground North Carolina

In [52]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="North Carolina"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("North Carolina")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/2366669228.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


North Carolina


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-22,16661,Kamala Harris,47.959028,0.05,19
1,2024-10-22,16651,Donald Trump,48.268157,0.05,19
2,2024-10-20,16661,Kamala Harris,47.919516,0.05,19
3,2024-10-20,16651,Donald Trump,48.140812,0.05,19
4,2024-10-18,16661,Kamala Harris,47.928028,0.05,19
5,2024-10-18,16651,Donald Trump,48.150873,0.05,19
6,2024-10-17,16661,Kamala Harris,47.876739,0.05,19
7,2024-10-17,16651,Donald Trump,48.172224,0.05,19
8,2024-10-15,16661,Kamala Harris,47.870283,0.05,19
9,2024-10-15,16651,Donald Trump,48.143314,0.05,19


In [53]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('North Carolina')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (North Carolina)"
)

# Display the final chart
final_chart.display()


North Carolina
     end_date candidate_name  weighted_average_pct
69 2024-10-22  Kamala Harris             47.903601
68 2024-10-22   Donald Trump             48.517826
67 2024-10-20  Kamala Harris             47.896923
66 2024-10-20   Donald Trump             48.356765
65 2024-10-18  Kamala Harris             48.014042
64 2024-10-18   Donald Trump             48.350837
63 2024-10-17  Kamala Harris             47.983882
62 2024-10-17   Donald Trump             48.316912
61 2024-10-15  Kamala Harris             47.962065
60 2024-10-15   Donald Trump             48.226932


alt.LayerChart(...)

In [54]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RMSE for Kamala Harris: 0.3008195181596195
Final forecast for Kamala Harris on Nov 5, 2024: 47.97%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


alt.LayerChart(...)

RMSE for Donald Trump: 0.12240204596611776
Final forecast for Donald Trump on Nov 5, 2024: 48.59%

Final Predictions for November 5, 2024:
Kamala Harris: 47.97%
Donald Trump: 48.59%


#### Boxplots of House Effects distributions by Pollster and Candidate

In [55]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### Battleground Georgia

In [56]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Georgia"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Georgia")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/1092981571.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Georgia


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-22,16661,Kamala Harris,47.315918,0.05,19
1,2024-10-22,16651,Donald Trump,48.891869,0.05,19
2,2024-10-20,16661,Kamala Harris,47.243570,0.05,19
3,2024-10-20,16651,Donald Trump,48.820349,0.05,19
4,2024-10-18,16661,Kamala Harris,47.228389,0.05,19
5,2024-10-18,16651,Donald Trump,48.826746,0.05,19
6,2024-10-17,16661,Kamala Harris,47.247717,0.05,19
7,2024-10-17,16651,Donald Trump,48.833548,0.05,19
8,2024-10-16,16661,Kamala Harris,47.262154,0.05,19
9,2024-10-16,16651,Donald Trump,48.821033,0.05,19


In [57]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Georgia')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Georgia)"
)

# Display the final chart
final_chart.display()


Georgia
     end_date candidate_name  weighted_average_pct
59 2024-10-22  Kamala Harris             47.383014
58 2024-10-22   Donald Trump             48.996932
57 2024-10-20  Kamala Harris             47.254992
56 2024-10-20   Donald Trump             49.016407
55 2024-10-18  Kamala Harris             47.202440
54 2024-10-18   Donald Trump             49.058334
53 2024-10-17  Kamala Harris             47.199873
52 2024-10-17   Donald Trump             49.052824
51 2024-10-16  Kamala Harris             47.191156
50 2024-10-16   Donald Trump             49.079163


alt.LayerChart(...)

In [58]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RMSE for Kamala Harris: 0.17732887756946808
Final forecast for Kamala Harris on Nov 5, 2024: 47.54%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


alt.LayerChart(...)

RMSE for Donald Trump: 0.12970845111838922
Final forecast for Donald Trump on Nov 5, 2024: 48.94%

Final Predictions for November 5, 2024:
Kamala Harris: 47.54%
Donald Trump: 48.94%


 #### Boxplots of House Effects distributions by Pollster and Candidate

In [59]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### Battleground Florida

In [60]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Florida"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Florida")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/3086076348.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Florida


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-25,16661,Kamala Harris,44.270677,0.05,19
1,2024-10-25,16651,Donald Trump,50.842916,0.05,19
2,2024-10-20,16661,Kamala Harris,44.253226,0.05,19
3,2024-10-20,16651,Donald Trump,50.634168,0.05,19
4,2024-10-18,16661,Kamala Harris,44.336573,0.05,19
5,2024-10-18,16651,Donald Trump,50.508946,0.05,19
6,2024-10-17,16661,Kamala Harris,44.736989,0.05,19
7,2024-10-17,16651,Donald Trump,50.166339,0.05,19
8,2024-10-14,16661,Kamala Harris,44.811730,0.05,19
9,2024-10-14,16651,Donald Trump,50.024031,0.05,19


In [61]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Florida')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Florida)"
)

# Display the final chart
final_chart.display()


Florida
     end_date candidate_name  weighted_average_pct
41 2024-10-25  Kamala Harris             44.584101
40 2024-10-25   Donald Trump             51.434304
39 2024-10-20  Kamala Harris             44.477758
38 2024-10-20   Donald Trump             51.142634
37 2024-10-18  Kamala Harris             44.404879
36 2024-10-18   Donald Trump             50.979701
35 2024-10-17  Kamala Harris             44.179802
34 2024-10-17   Donald Trump             50.829641
33 2024-10-14  Kamala Harris             43.851684
32 2024-10-14   Donald Trump             50.900016


alt.LayerChart(...)

In [62]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RMSE for Kamala Harris: 0.6421767862445066
Final forecast for Kamala Harris on Nov 5, 2024: 44.39%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


alt.LayerChart(...)

RMSE for Donald Trump: 0.5696288371520539
Final forecast for Donald Trump on Nov 5, 2024: 51.58%

Final Predictions for November 5, 2024:
Kamala Harris: 44.39%
Donald Trump: 51.58%


#### Boxplots of House Effects distributions by Pollster and Candidate

In [63]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### Battleground Arizona

In [64]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Arizona"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Arizona")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/2549904380.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Arizona


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-22,16661,Kamala Harris,47.283941,0.05,19
1,2024-10-22,16651,Donald Trump,48.933190,0.05,19
2,2024-10-20,16661,Kamala Harris,47.191526,0.05,19
3,2024-10-20,16651,Donald Trump,48.849385,0.05,19
4,2024-10-18,16661,Kamala Harris,47.123808,0.05,19
5,2024-10-18,16651,Donald Trump,48.804241,0.05,19
6,2024-10-17,16661,Kamala Harris,47.125581,0.05,19
7,2024-10-17,16651,Donald Trump,48.738664,0.05,19
8,2024-10-16,16661,Kamala Harris,47.123708,0.05,19
9,2024-10-16,16651,Donald Trump,48.671772,0.05,19


In [65]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Arizona')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Arizona)"
)

# Display the final chart
final_chart.display()


Arizona
     end_date candidate_name  weighted_average_pct
63 2024-10-22  Kamala Harris             47.317590
62 2024-10-22   Donald Trump             49.033230
61 2024-10-20  Kamala Harris             47.211134
60 2024-10-20   Donald Trump             49.165646
59 2024-10-18  Kamala Harris             47.029311
58 2024-10-18   Donald Trump             49.258318
57 2024-10-17  Kamala Harris             46.992468
56 2024-10-17   Donald Trump             49.212333
55 2024-10-16  Kamala Harris             46.958474
54 2024-10-16   Donald Trump             49.168716


alt.LayerChart(...)

In [66]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

RMSE for Kamala Harris: 0.2117228107229367
Final forecast for Kamala Harris on Nov 5, 2024: 47.67%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

RMSE for Donald Trump: 0.19506513278354962
Final forecast for Donald Trump on Nov 5, 2024: 49.08%

Final Predictions for November 5, 2024:
Kamala Harris: 47.67%
Donald Trump: 49.08%


#### Boxplots of House Effects distributions by Pollster and Candidate

In [67]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### Battleground Nevada

In [68]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Nevada"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Nevada")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/583348984.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Nevada


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-22,16661,Kamala Harris,48.013887,0.05,19
1,2024-10-22,16651,Donald Trump,47.483114,0.05,19
2,2024-10-20,16661,Kamala Harris,48.000535,0.05,19
3,2024-10-20,16651,Donald Trump,47.481918,0.05,19
4,2024-10-18,16661,Kamala Harris,48.039636,0.05,19
5,2024-10-18,16651,Donald Trump,47.428475,0.05,19
6,2024-10-17,16661,Kamala Harris,48.105271,0.05,19
7,2024-10-17,16651,Donald Trump,47.392030,0.05,19
8,2024-10-15,16661,Kamala Harris,48.237698,0.05,19
9,2024-10-15,16651,Donald Trump,47.359698,0.05,19


In [69]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Nevada')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Nevada)"
)

# Display the final chart
final_chart.display()


Nevada
     end_date candidate_name  weighted_average_pct
47 2024-10-22  Kamala Harris             47.784346
46 2024-10-22   Donald Trump             47.700248
45 2024-10-20  Kamala Harris             47.801517
44 2024-10-20   Donald Trump             47.720098
43 2024-10-18  Kamala Harris             47.904990
42 2024-10-18   Donald Trump             47.527512
41 2024-10-17  Kamala Harris             47.956731
40 2024-10-17   Donald Trump             47.481496
39 2024-10-15  Kamala Harris             47.991389
38 2024-10-15   Donald Trump             47.441349


alt.LayerChart(...)

In [70]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_aggregated is already defined
# Make sure 'end_date' is in datetime format
df_aggregated['end_date'] = pd.to_datetime(df_aggregated['end_date'])

# Define a new dataframe to store the LOESS smoothed data
loess_data = pd.DataFrame()

# Perform LOESS smoothing for each candidate
for candidate in df_aggregated['candidate_name'].unique():
    # Filter the data for each candidate
    candidate_df = df_aggregated[df_aggregated['candidate_name'] == candidate]
    
    # Perform LOESS smoothing (use frac to control the smoothing level)
    loess_smoothed = lowess(candidate_df['weighted_average_pct'], 
                            candidate_df['end_date'].apply(lambda x: x.timestamp()), 
                            frac=0.3)  # Adjust frac as needed
    
    # Create a dataframe from the smoothed data
    smoothed_df = pd.DataFrame({
        'end_date': pd.to_datetime(loess_smoothed[:, 0], unit='s'),
        'weighted_average_pct': loess_smoothed[:, 1],
        'candidate_name': candidate
    })
    
    # Append to the main loess_data dataframe
    loess_data = pd.concat([loess_data, smoothed_df], ignore_index=True)

# Now let's fit an ARIMA model using the LOESS smoothed data
candidate_forecasts = {}
# Store the final prediction for each candidate
final_predictions = {}
# Forecast up to November 5, 2024
forecast_end_date = pd.Timestamp('2024-11-05')

# Loop through each candidate and apply ARIMA
for candidate in loess_data['candidate_name'].unique():
    # Filter the LOESS smoothed data for this candidate
    candidate_loess_df = loess_data[loess_data['candidate_name'] == candidate].sort_values(by='end_date')
    
    # Set 'end_date' as the index for ARIMA (must be a time series index)
    candidate_loess_df.set_index('end_date', inplace=True)

    # Calculate the number of days to forecast (from the last date to Nov 5, 2024)
    last_date = candidate_loess_df.index[-1]
    days_to_forecast = (forecast_end_date - last_date).days
    
    # Fit ARIMA model (you can tune the order=(p, d, q))
    model = ARIMA(candidate_loess_df['weighted_average_pct'], order=(1, 1, 1))
    model_fit = model.fit()

    # In-sample forecast (fitted values) and out-of-sample forecast
    candidate_loess_df['fitted'] = model_fit.fittedvalues
    
    # Forecast up to November 5, 2024
    forecast = model_fit.forecast(steps=days_to_forecast)
    
    # Store the forecasted results for this candidate
    candidate_forecasts[candidate] = forecast
    
    # Generate forecast dates for visualization
    forecast_dates = pd.date_range(candidate_loess_df.index[-1], periods=days_to_forecast + 1, freq='D')[1:]
    
    forecast_df = pd.DataFrame({
        'end_date': forecast_dates,
        'forecast': forecast,
        'candidate_name': candidate
    })
    
    # Reset index for easier plotting
    candidate_loess_df = candidate_loess_df.reset_index()

    # Altair plot for the actual, smoothed, and forecast data
    loess_chart = alt.Chart(candidate_loess_df).mark_line(color='blue', size=3).encode(
        x='end_date:T',
        y='weighted_average_pct:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Date'),
                 alt.Tooltip('weighted_average_pct:Q', title='Smoothed Data')]
    ).properties(title=f'ARIMA Forecast on LOESS Smoothed Data for {candidate}')

    fitted_chart = alt.Chart(candidate_loess_df).mark_line(color='orange', size=2).encode(
        x='end_date:T',
        y='fitted:Q',
        tooltip=['fitted:Q']
    )

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green', size=2).encode(
        x='end_date:T',
        y='forecast:Q',
        tooltip=[alt.Tooltip('end_date:T', title='Forecast Date'),
                 alt.Tooltip('forecast:Q', title='Forecasted Value')]
    )
    
    # Combine the charts and make the graph larger
    combined_chart = (loess_chart + fitted_chart + forecast_chart).properties(
        width=800,  # Set larger width
        height=400  # Set larger height
    )
    
    # Display the chart
    combined_chart.display()

    # Optionally, calculate the RMSE of the model
    rmse = np.sqrt(mean_squared_error(candidate_loess_df['weighted_average_pct'][1:], candidate_loess_df['fitted'][1:]))
    print(f"RMSE for {candidate}: {rmse}")
    # Extract the forecasted value for Nov 5, 2024
    nov_5_prediction = forecast_df[forecast_df['end_date'] == forecast_end_date]['forecast'].values[0]
    
    # Store the prediction in the final_predictions dictionary
    final_predictions[candidate] = nov_5_prediction

    # Print the final prediction for the candidate
    print(f"Final forecast for {candidate} on Nov 5, 2024: {nov_5_prediction:.2f}%")
    
# Optionally, print the final predictions for both candidates in a clear format
print("\nFinal Predictions for November 5, 2024:")
for candidate, prediction in final_predictions.items():
    print(f"{candidate}: {prediction:.2f}%")    


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

RMSE for Kamala Harris: 0.9303633874991206
Final forecast for Kamala Harris on Nov 5, 2024: 47.91%


/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/christopher/ppoll_jupyterlab/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Predicti

alt.LayerChart(...)

RMSE for Donald Trump: 0.3371263138050677
Final forecast for Donald Trump on Nov 5, 2024: 48.42%

Final Predictions for November 5, 2024:
Kamala Harris: 47.91%
Donald Trump: 48.42%


#### Boxplots of House Effects distributions by Pollster and Candidate

In [71]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### State of Minnesota

In [72]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Minnesota"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Minnesota")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/556234786.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Minnesota


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-14,16661,Kamala Harris,50.024647,0.05,19
1,2024-10-14,16651,Donald Trump,43.642958,0.05,19
2,2024-10-02,16661,Kamala Harris,49.549124,0.05,19
3,2024-10-02,16651,Donald Trump,43.635015,0.05,19
4,2024-09-26,16661,Kamala Harris,49.543630,0.05,19
5,2024-09-26,16651,Donald Trump,43.745786,0.05,19
6,2024-09-19,16661,Kamala Harris,49.488463,0.05,19
7,2024-09-19,16651,Donald Trump,43.552888,0.05,19
8,2024-09-18,16661,Kamala Harris,49.632818,0.05,19
9,2024-09-18,16651,Donald Trump,44.026935,0.05,19


In [73]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Minnesota')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Minnesota)"
)

# Display the final chart
final_chart.display()


Minnesota
     end_date candidate_name  weighted_average_pct
23 2024-10-14  Kamala Harris             50.484516
22 2024-10-14   Donald Trump             43.428514
21 2024-10-02  Kamala Harris             49.855484
20 2024-10-02   Donald Trump             43.311427
19 2024-09-26  Kamala Harris             49.548850
18 2024-09-26   Donald Trump             43.619304
17 2024-09-19  Kamala Harris             49.470230
16 2024-09-19   Donald Trump             43.684761
15 2024-09-18  Kamala Harris             49.491844
14 2024-09-18   Donald Trump             43.868985


alt.LayerChart(...)

In [74]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()

# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### State of Texas

In [75]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Texas"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Texas")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/115735657.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Texas


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-26,16661,Kamala Harris,44.857479,0.05,19
1,2024-10-26,16651,Donald Trump,50.983031,0.05,19
2,2024-10-21,16661,Kamala Harris,44.866955,0.05,19
3,2024-10-21,16651,Donald Trump,50.643697,0.05,19
4,2024-10-15,16661,Kamala Harris,44.766131,0.05,19
5,2024-10-15,16651,Donald Trump,50.874626,0.05,19
6,2024-10-10,16661,Kamala Harris,44.844448,0.05,19
7,2024-10-10,16651,Donald Trump,50.860361,0.05,19
8,2024-10-07,16661,Kamala Harris,44.804203,0.05,19
9,2024-10-07,16651,Donald Trump,50.723744,0.05,19


In [76]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Texas')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Texas)"
)

# Display the final chart
final_chart.display()


Texas
     end_date candidate_name  weighted_average_pct
25 2024-10-26  Kamala Harris             44.865551
24 2024-10-26   Donald Trump             51.320451
23 2024-10-21  Kamala Harris             44.869319
22 2024-10-21   Donald Trump             50.655899
21 2024-10-15  Kamala Harris             44.865583
20 2024-10-15   Donald Trump             50.726239
19 2024-10-10  Kamala Harris             44.579204
18 2024-10-10   Donald Trump             50.581573
17 2024-10-07  Kamala Harris             44.276696
16 2024-10-07   Donald Trump             50.236269


alt.LayerChart(...)

In [77]:
# Group by 'pollster' and calculate summary statistics for 'house_effect_data'
he_agg = house_effect_data.groupby(['end_date','pollster','candidate_name']).agg({
    'house_effect': 'mean'
}).reset_index()
# Define a custom color scale for specific candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'],  # Candidates
                        range=['blue', 'red'])  # Corresponding colors

# Create a boxplot with Altair, faceted by both pollster and candidate_name
boxplot = alt.Chart(he_agg).mark_boxplot().encode(
    x='state:N',  # Categorical data for states on the x-axis
    y='house_effect:Q',  # Quantitative data for house_effect on the y-axis
    color=alt.Color('candidate_name:N', scale=color_scale) # Color by candidate_name
).facet(
    row='candidate_name:N',  # Facet by pollster
    column='pollster:N'  # Facet by candidate_name
).properties(
    title='House Effect Bias Distribution by State, Pollster, and Candidate'
).configure_facet(
    spacing=100  # Adjust space between the facets (columns and rows)
).resolve_scale(
    y='independent'  # Allow each facet to have its own independent y-scale
)

boxplot.show()

alt.FacetChart(...)

#### State of Virginia

In [78]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = .05
avg_window = 9
weighted_averages = []
# Initialize lambda and average_window settings
lambda_values = [.05,.15,.35,.55,.65]  # 5 equal subdivisions between 0 and 1
average_windows = [19, 9]

# Define the two candidate IDs of interest
candidate_ids = [16661, 16651]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_not_na_clean_sorted_cutoff = df_state_not_na_clean_sorted[df_state_not_na_clean_sorted['end_date'] >= cutoff_date].copy()


df_mi = df_state_not_na_clean_sorted_cutoff[df_state_not_na_clean_sorted_cutoff['state']=="Virginia"].copy()
df_mi = df_mi[df_mi['weight_score'] >= 0.4].copy() #using data from F rated pollsters and above
df_mi = df_mi[df_mi['population'] == 'lv'].copy() #gather likely voter data
#print(df_mi)
#Recording house effect data for visualization reuse
# Step 1: Initialize an empty DataFrame outside the loop
columns = ['end_date','pollster', 'state', 'candidate_name', 'house_effect']
house_effect_data = pd.DataFrame(columns=columns)
#end recording house effect data...
for lambda_value in lambda_values:
    for avg_window in average_windows:
        # Iterate through each unique end date
        for current_date in df_mi['end_date'].unique():
            
            # Step 2: Filter the data for polls on or before the current end date
            current_data = df_mi[df_mi['end_date'] <= current_date].copy()
            
            # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
            current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
            
            # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)  
            current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])
        
            # Check for NaN entries in the 'weight_time' column
            #nan_weight_time = current_data['weight_time'].isna().sum()
            
            # Print the result
            #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")
        
            # Step 2: Check for infinite values in the involved columns
            #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
            #print(f"Number of infinite values:\n{inf_check}")
            
            # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
            for candidate in candidate_ids:
                
                # Step 7: Filter data for the specific candidate
                candidate_data = current_data[current_data['candidate_id'] == candidate].copy()
                # Step 1: Group by 'question_id' and count occurrences
                #question_id_counts = candidate_data['question_id'].value_counts()
                
                # Step 2: Get the 'question_id' values that occur more than 2 times
                #exclude_question_ids = question_id_counts[question_id_counts > 2].index
                
                # Step 3: Filter the DataFrame to exclude 'question_id' values with more than 2 occurrences
                #candidate_data = candidate_data[~candidate_data['question_id'].isin(exclude_question_ids)].copy()
                #  We select H2H if avaible and only use non H2H if H2H is not available
        
                # Step 1: Group by 'created_at' and 'question_id' and count occurrences
                grouped = candidate_data.groupby(['created_at', 'question_id']).size().reset_index(name='count')
                
                # Step 2: Initialize an empty DataFrame to store results
                result_df = pd.DataFrame()
    
                # Step 3: Iterate through each 'created_at' date group
                for date, group in grouped.groupby('created_at'):
                    # Step 4: Filter for question_ids with count == 2
                    question_ids_with_count_2 = group[group['count'] == 2]
                    
                    if not question_ids_with_count_2.empty:
                        # If question_ids with count 2 exist, include only them
                        selected_question_ids = question_ids_with_count_2['question_id']
                    else:
                        # If no question_ids with count 2 exist, include all question_ids for that date
                        selected_question_ids = group['question_id']
                    
                    # Step 5: Filter the original DataFrame to include only selected question_ids for that date
                    filtered_df = candidate_data[(candidate_data['created_at'] == date) & (candidate_data['question_id'].isin(selected_question_ids))]
                    
                    # Step 6: Append the filtered DataFrame to the result
                    result_df = pd.concat([result_df, filtered_df])
                candidate_data = result_df.copy()
                ## start pollster frequency weighting

                # Define 30-day intervals using the 'days_past_index' column
                interval_ranges = [(0, 30), (31, 60), (61, 90)]
                
                # Initialize a dictionary to store results for each interval
                interval_pollster_counts_days_index = {}
                
                # Iterate over each interval
                for start, end in interval_ranges:
                    # Filter the data within the interval using 'days_past_index'
                    interval_data = candidate_data[(candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)]
                    #interval_data = interval_data[interval_data['candidate_id']==16661]
                    # Group by 'pollster' and count the occurrences
                    pollster_count = interval_data.groupby(['pollster','candidate_id'])['pollster'].count()
                    #print(pollster_count.head(5))
                    # Calculate weight_pollster_frequency = 1 / pollster_count
                    weight_pollster_frequency = 1 / pollster_count
                    
                    # Store the result in the dictionary with interval as the key
                    interval_pollster_counts_days_index[f'{start} to {end}'] = weight_pollster_frequency
                
                # Convert the results into a DataFrame and display it
                pollster_weights_days_index_df = pd.DataFrame(interval_pollster_counts_days_index)
                #print(pollster_weights_days_index_df.head(5))
                # Initialize a new column in the original dataframe to store the joined weight_pollster_frequency
                candidate_data['weight_pollster_frequency'] = None
                
                # Iterate over each interval range and merge/join the weight_pollster_frequency values
                for interval_name, interval_data in pollster_weights_days_index_df.items():
                    # Determine the appropriate interval (e.g., '0 to 30 days' corresponds to days_past_index between 0 and 30)
                    start, end = map(int, interval_name.split(' to '))
                    
                    # Filter the rows of candidate_data within the current interval range
                    interval_mask = (candidate_data['days_past_index'] >= start) & (candidate_data['days_past_index'] <= end)
                    
                    # Filter the rows for the current interval
                    candidate_data_interval = candidate_data[interval_mask]
                    
                    # Merge with the corresponding interval from pollster_candidate_weights_df on 'pollster' and 'candidate_id'
                    merged_data = pd.merge(candidate_data_interval, 
                                           pollster_weights_days_index_df[[interval_name]], 
                                           left_on=['pollster', 'candidate_id'], 
                                           right_index=True, 
                                           how='left')
                    
                    # Update the 'weight_pollster_frequency' column in the original candidate_data with the merged values
                    candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = merged_data[interval_name]
                    # Merge the corresponding weight_pollster_frequency values from pollster_weights_days_index_df based on 'pollster'
                    #candidate_data.loc[interval_mask, 'weight_pollster_frequency'] = candidate_data.loc[interval_mask, 'pollster'].map(pollster_weights_days_index_df[interval_name])
                    
                ## end  pollster frequency weighting
                c_mean = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].mean() #gather mean for past 30 days of likely voters
        
                c_std = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] > 0)]['pct'].std() #gather standard deviation for past 30 days of likely voters
                #print(candidate)
                #print(c_mean)
                #print(c_std)
                #print(candidate_data['pct'].iloc[0])
                #print(abs((candidate_data['pct'].iloc[0]-c_mean)/c_std))
                candidate_data['zscores'] = abs((candidate_data['pct']-c_mean)/c_std)
                #print(candidate_data['zscores'])
                #df['z_score'].apply(lambda z: z ** 2 if abs(z) > 1 else z)
                candidate_data['weight_outlier'] = np.exp(-1*candidate_data['zscores'])#.apply(lambda z: z**2 if abs(z) > 2.5 else (z**1.75 if abs(z) > 2.0 else (z**1.5 if abs(z) > 1.5 else z))))
                #lets grab house effect data
                # Group the data by 'pollster' and calculate the mean of 'pct' for each pollstera
                #Start House effect data computation#
                #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
                candidate_data.loc[:, 'w_i'] = (
                    candidate_data['weight_mode'] * candidate_data['weight_sample'] *
                    candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']*
                    candidate_data['weight_pollster_frequency']
                )
                #c_mean2data = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]
                #c_mean2 = np.average(c_mean2data['pct'], weights=c_mean2data['w_i'])
                #c_mean2 = candidate_data[(candidate_data['days_past_index'] < avg_window) & (candidate_data['days_past_index'] >= 0)]['pct'].mean() 
                ##c_mean_by_pollster = candidate_data.groupby('pollster_id')['pct'].mean()
                
                    # Create an empty dictionary to store the results
                pollster_means = {}
            
                # Group the data by 'pollster_id'
                grouped = candidate_data.groupby('pollster_id')
            
                # Loop through each group (i.e., each pollster)
                for pollster_id, group in grouped:
                    # Find the most recent 'end_date' for this pollster
                    last_end_date_index = group['days_past_index'].min()
                    
                    # Calculate the start date for the filtering (last_end_date - x_days)
                    start_date_index = last_end_date_index+avg_window
            
                    # Filter the group's data to only include rows between start_date and last_end_date
                    filtered_group = group[(group['days_past_index'] >= last_end_date_index) & (group['days_past_index'] <= start_date_index)]
                    
                    # Calculate the mean of 'pct' for this filtered group
                    pollster_mean = filtered_group['pct'].mean()
                    c_mean2data = candidate_data[(candidate_data['days_past_index'] >= last_end_date_index) & (candidate_data['days_past_index'] <= start_date_index)]
                    c_mean2 = c_mean2data['pct'].mean() #np.average(c_mean2data['pct'], weights=c_mean2data['w_i']) #c_mean2data['pct'].mean()
                    # Store the result in the dictionary
                    house_effect = pollster_mean - c_mean2
                    pollster_means[pollster_id] = house_effect
            
                # Convert the dictionary to a pandas Series for easier handling
                c_mean_by_pollster = pd.Series(pollster_means)

                # Convert to a DataFrame for easier manipulation
                c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
                # Rename the columns for clarity
                c_mean_by_pollster_df.columns = ['pollster_id', 'house_effect']
                # Check for NaN values in the entire DataFrame
                #nan_check = c_mean_by_pollster_df.isna().sum()
                #print(c_mean)
                
                # Display the result
                #print(nan_check)
                # Calculate the house effect for each pollster (difference from overall mean)
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['mean_pct'] - c_mean2
                # Fill NaN values with 0 in the 'house_effect' column
                #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
                
                #c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
                #print(c_mean_by_pollster_df.head())
                # Merge the house effect back into the original data
                candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
                
                candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
                #print(candidate_data.head(10))
                # Apply the house effect to adjust the 'pct' values
                candidate_data['adjusted_pct'] = candidate_data['pct'] - candidate_data['house_effect']
                #start saving house effect data for boxplots
                new_data = pd.DataFrame({
                    'end_date': current_date,
                    'pollster': candidate_data['pollster'],  # Existing pollster data
                    'state': candidate_data['state'],  # Existing state data
                    'candidate_name': candidate_data['candidate_name'],  # Adding candidate_name
                    'house_effect': candidate_data['house_effect']  # Existing house effect data
                })
                
                # Append new data to the main DataFrame
                house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)
                #end saving house effect data for box plots
                #print(candidate_data['adjusted_pct'].head(10))
                # Step 8: Compute the total weight 'w_i' for each poll

                # check for NaN entries in the w_i column
                #nan_weight_i = candidate_data['w_i'].isna().sum()
                #print result
                #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
                
                # Step 3: Identify rows with NaN in w_i to examine their individual values
                #nan_rows = candidate_data[candidate_data['w_i'].isna()]
                #print("Rows producing NaN in 'w_i':")
                #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
                
                # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
                weighted_average_pct = np.average(candidate_data['adjusted_pct'], weights=candidate_data['w_i'])
                
                # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
                if not candidate_data.empty:
                    candidate_name = candidate_data['candidate_name'].iloc[0]
                    weighted_averages.append({
                        'end_date': current_date,
                        'candidate_id': candidate,
                        'candidate_name': candidate_name,
                        'weighted_average_pct': weighted_average_pct,
                        'lambda': lambda_value,
                        'average_windows': avg_window
                    })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
print("Virginia")
df_weighted_averages.head(10)       


/tmp/ipykernel_76441/1953217500.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  house_effect_data = pd.concat([house_effect_data, new_data], ignore_index=True)


Virginia


,end_date,candidate_id,candidate_name,weighted_average_pct,lambda,average_windows
0,2024-10-23,16661,Kamala Harris,NaN,0.05,19
1,2024-10-23,16651,Donald Trump,NaN,0.05,19
2,2024-10-04,16661,Kamala Harris,51.611058,0.05,19
3,2024-10-04,16651,Donald Trump,43.556177,0.05,19
4,2024-09-24,16661,Kamala Harris,51.039675,0.05,19
5,2024-09-24,16651,Donald Trump,42.735031,0.05,19
6,2024-09-18,16661,Kamala Harris,51.005104,0.05,19
7,2024-09-18,16651,Donald Trump,42.216649,0.05,19
8,2024-09-08,16661,Kamala Harris,NaN,0.05,19
9,2024-09-08,16651,Donald Trump,NaN,0.05,19


In [79]:
import altair as alt

# Aggregating the average for groupby 'end_date' and 'candidate_name'
df_aggregated = df_weighted_averages.groupby(['end_date', 'candidate_name']).agg({
    'weighted_average_pct': 'mean'
}).reset_index().sort_values(by='end_date',ascending=False)
print('Virginia')
print(df_aggregated.head(10))
# Define the custom color scale for the candidates
color_scale = alt.Scale(domain=['Kamala Harris', 'Donald Trump'], range=['blue', 'red'])

# Scatter plot with distinct shapes for each candidate
scatter = alt.Chart(df_aggregated).mark_point(size=60).encode(
    x=alt.X('end_date:T', title='End Date'),
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)', scale=alt.Scale(domain=[41, 52])),  # Restricted vertical axis
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    shape=alt.Shape('candidate_name:N', title='Candidate'),  # Different shapes for each candidate
    tooltip=[alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('weighted_average_pct:Q', title='Weighted Average (%)')]
).properties(
    width=800,
    height=400
)

# LOESS smoothing for each candidate, applied to the aggregated data
loess = alt.Chart(df_aggregated).transform_loess(
    'end_date', 'weighted_average_pct', groupby=['candidate_name']
).mark_line(size=3).encode(
    x='end_date:T',
    y=alt.Y('weighted_average_pct:Q', title='Weighted Average (%)'),
    color=alt.Color('candidate_name:N', scale=color_scale, title='Candidate'),  # Custom color scale for candidates
    tooltip=[alt.Tooltip('candidate_name:N', title='Candidate'), alt.Tooltip('end_date:T', title='Date'), alt.Tooltip('weighted_average_pct:Q', title='Smoothed Average (%)')]
)

# Layer scatter plot and LOESS smoothing
final_chart = alt.layer(scatter, loess).properties(
    title="Aggregated Weighted Averages and LOESS Curves for Each Candidate (Virginia)"
)

# Display the final chart
final_chart.display()


Virginia
     end_date candidate_name  weighted_average_pct
10 2024-10-23   Donald Trump                   NaN
11 2024-10-23  Kamala Harris                   NaN
8  2024-10-04   Donald Trump             43.755619
9  2024-10-04  Kamala Harris             51.764314
6  2024-09-24   Donald Trump             43.303597
7  2024-09-24  Kamala Harris             51.117594
4  2024-09-18   Donald Trump             42.828839
5  2024-09-18  Kamala Harris             50.587070
2  2024-09-08   Donald Trump                   NaN
3  2024-09-08  Kamala Harris                   NaN


alt.LayerChart(...)